In [14]:
# 安装工具链
%pip install uv
!uv tool install "huggingface_hub[cli]"


`huggingface-hub[cli]` is already installed


In [15]:
# ✅ Step 1: 获取最近n个提交
import requests
import ipywidgets as widgets
from IPython.display import display
import subprocess

# 仓库信息
repo = "index-tts/index-tts"
commits_url = f"https://api.github.com/repos/{repo}/commits"
response = requests.get(commits_url)
commits = response.json()

# 提取前两个提交 SHA 和信息
options = []
for commit in commits[:3]:
    sha = commit["sha"]
    message = commit["commit"]["message"]
    options.append((f"{message[:60]}... ({sha[:7]})", sha))  # 显示前60字+SHA前7位

# ✅ Step 2: 创建下拉菜单和按钮
dropdown = widgets.Dropdown(
    options=options,
    description='选择版本:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

button = widgets.Button(description="下载选定版本", button_style='success')
output = widgets.Output()

# ✅ Step 3: 下载并解压所选版本
def download_selected_version(sha):
    url = f"https://github.com/index-tts/index-tts/archive/{sha}.zip"
    subprocess.run(["wget", "-O", "main.zip", url])
    subprocess.run(["unzip", "main.zip"])
    subprocess.run(["mv", f"index-tts-{sha}", "index-tts"])
    subprocess.run(["rm", "main.zip"])

def on_button_clicked(b):
    with output:
        output.clear_output()
        print(f"正在下载版本：{dropdown.label}")
        download_selected_version(dropdown.value)
        print("✅ 下载完成，已解压到 index-tts 文件夹")

button.on_click(on_button_clicked)

# ✅ Step 4: 显示控件
display(dropdown, button, output)


Dropdown(description='选择版本:', layout=Layout(width='80%'), options=(('Merge pull request #537 from pandalee99/p…

Button(button_style='success', description='下载选定版本', style=ButtonStyle())

Output()

In [18]:
# 安装依赖
!cd /content/index-tts/index-tts && uv sync --all-extras

Using CPython 3.10.12 interpreter at: /usr/bin/python3.10
Creating virtual environment at: .venv
Resolved 190 packages in 52ms
Prepared 178 packages in 1m 32s
Installed 178 packages in 1.83s
Prepared 1 package without build isolation in 33.59s
Installed 1 package in 19ms
 + absl-py==2.3.1
 + accelerate==1.8.1
 + aiofiles==24.1.0
 + annotated-types==0.7.0
 + antlr4-python3-runtime==4.9.3
 + anyio==4.10.0
 + argbind==0.3.9
 + asttokens==3.0.0
 + audioread==3.0.1
 + brotli==1.1.0
 + cachetools==5.5.2
 + certifi==2025.8.3
 + cffi==2.0.0
 + charset-normalizer==3.4.3
 + click==8.2.1
 + cn2an==0.5.22
 + contourpy==1.3.2
 + cycler==0.12.1
 + cython==3.0.7
 + decorator==5.2.1
 + deepspeed==0.17.1
 + descript-audiotools==0.7.2
 + distance==0.1.3
 + docstring-parser==0.17.0
 + einops==0.8.1
 + exceptiongroup==1.3.0
 + executing==2.2.1
 + fastapi==0.116.2
 + ffmpeg-python==0.2.0
 + ffmpy==0.6.1
 + filelock==3.19.1
 + fire==0.7.1
 + flatten-dict==0.4.2
 + fonttools==4.60.0
 + fsspec==2025.9.0
 + fu

In [ ]:
# 下载模型 checkpoints
!hf download IndexTeam/IndexTTS-2 --local-dir=/content/index-tts/checkpoints

In [ ]:
# 为了直接利用 Gradio 的内网穿透功能
# 需要对 index—tts/webui.py 进行修改

# 如果因为 IndexTTS 官方 GitHub 仓库的更改造成本代码不可用
# 还可以手动修改 webui.py ，在 launch 函数调用中加上 `share = True`


%cd /content/index-tts

launch_line = 'demo.launch(server_name=cmd_args.host, server_port=cmd_args.port)'
launch_line_with_share = 'demo.launch(server_name=cmd_args.host, server_port=cmd_args.port, share=True)'

with open("webui.py", encoding='utf8') as f:
  webui_with_share = []
  for line in f.readlines():
    if line.strip() == launch_line:
      webui_with_share.append(line.replace(launch_line, launch_line_with_share))
    else:
      webui_with_share.append(line)

  with open("webui_with_share.py", 'w', encoding='utf8') as f:
    f.writelines(webui_with_share)

/content/index-tts


In [ ]:
# 启动

%cd /content/index-tts/
!uv run webui_with_share.py